### 1. Paquetes necesarios

In [ ]:
%pip install llama-index-llms-huggingface
!pip install llama-index
!pip install transformers accelerate bitsandbytes
!pip install llama-index-embeddings-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 92.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 69.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 67.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 96.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
import torch
from transformers import BitsAndBytesConfig
from llama_index.core import PromptTemplate
from llama_index.llms.huggingface import HuggingFaceLLM

### 2. Carga del modelo desde HuggingFace

- Como es una prueba hemos cargado mistral en vez de mixtral por simplicidad
- En el system prompt hay que poner mas cosas, por ejemplo que no responda con información que no sea la que saca de los textos

In [ ]:
quantization_config = BitsAndBytesConfig( # configuración de cuantificación del modelo -> reduce tamaño y aumenta velocidad
    load_in_4bit=True,                    # carga el modelo en formato 4bits
    bnb_4bit_compute_dtype=torch.float16, # más configuraciones relacionadas con eso
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

llm = HuggingFaceLLM(                                                             # se carga mistral desde hugging face
    model_name="mistralai/Mistral-7B-Instruct-v0.1",                              # modelo
    tokenizer_name="mistralai/Mistral-7B-Instruct-v0.1",                          # tokenizador
    query_wrapper_prompt=PromptTemplate("<s>[INST] {query_str} [/INST] </s>\n"),  # plantilla para envolver las consultas al modelo, lo que ayuda a guiar al modelo para responder de una manera específica
    context_window=3900,                                                          # tamaño máximo de la ventana de contexto que el modelo puede considerar para cada predicción
    max_new_tokens=256,                                                           # tokens que el modelo puede generar
    model_kwargs={"quantization_config": quantization_config},                    # configuración de cuantificación
    generate_kwargs={"temperature": 0.2, "top_k": 5, "top_p": 0.95},              # parámetros para la generación de texto
    device_map="auto",                                                            # el modelo se distribuye automaticamente entre los dispositivos disponibles
    system_prompt="Always respond in Spanish."
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

Por simplicidad tambien se han cargado los embeddings BAAI/bge-small-en-v1.5, que son solo en inglés. Habría que usar otros que fueran en español, como por ejemplo BAAI/bge-m3

In [ ]:
from llama_index.core import ServiceContext

service_context = ServiceContext.from_defaults(llm=llm, embed_model="local:BAAI/bge-small-en-v1.5")

<ipython-input-4-83586933102f>:3: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(llm=llm, embed_model="local:BAAI/bge-small-en-v1.5")


config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Se cargan los datos. Deberían estar preprocesados ya

In [ ]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader("/data").load_data()

### 3. Indexación de los textos cargados

In [ ]:
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex.from_documents(documents, service_context=service_context)

### 4. Pruebas de preguntas

In [ ]:
from llama_index.core.response.notebook_utils import display_response

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [ ]:
query_engine = vector_index.as_query_engine(response_mode="compact")

response = query_engine.query("¿Cuál es la misión de las fuerzas armadas?")

display_response(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**`Final Response:`** The mission of the armed forces is not explicitly stated in the provided text.

In [ ]:
query_engine = vector_index.as_query_engine(response_mode="refine")

response = query_engine.query("¿Cuál es la misión de las fuerzas armadas?")

display_response(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**`Final Response:`** La misión de las fuerzas armadas es la defensa de la nación y la garantía de la seguridad de sus ciudadanos. Esto se logra a través de varios actos preliminares y posteriores al servicio de armas, como la organización de fuerzas, la preparación de armamento y equipamiento, la formación de personal militar y civil, y la ejecución de operaciones militares en caso de conflicto.

Las fuerzas armadas deben estar frente al enemigo cuando este se hallare notoriamente y constituyendo fuerza armada en el territorio, mar o aire declarado en estado de guerra o en operaciones de campaña, a una distancia que haga posible de modo inmediato el combate. Las fuerzas antiaéreas de los tres Ejércitos se consideran también al frente del enemigo mientras estén en situación de alerta. Las fuerzas navales lo estarán, además, siempre que se hallen desempeñando alguna misión de guerra.

Las unidades de los Ejércitos de

In [ ]:
query_engine = vector_index.as_query_engine(response_mode="compact")

response = query_engine.query("¿Cuál es la capital de España?")

display_response(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**`Final Response:`** La capital de España es Madrid.

In [ ]:
query_engine = vector_index.as_query_engine(response_mode="refine")

response = query_engine.query("¿Cuál es la capital de España?")

display_response(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**`Final Response:`** La capital de Aragón es Zaragoza.

In [ ]:
query_engine = vector_index.as_query_engine(response_mode="refine")

response = query_engine.query("¿Cuántos artículos tiene la ley orgánica?")

display_response(response)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:407: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_to

**`Final Response:`** La presente Ley tiene el carácter de ley orgánica, a excepción de los siguientes preceptos y disposiciones:

* Artículos 1 al 9, ambos inclusive, con excepción de los artículos 2 y 4, apartado 2, que sí tienen carácter orgánico.
* Artículo 15, apartados 4 y 5, excepto los apartados 1 a 3, que tienen carácter orgánico.
* Artículos 16 y 17.
* Artículo 18, excepto el apartado primero que tiene carácter orgánico.
* Artículos 19 al 40, excepto el artículo 32 y el artículo 33, que sí tienen carácter orgánico.
* Artículos 41 al 50 y artículos 52 y 53.
* Artículos 55 a 62, excepto los párrafos 1.º y 2.º del artículo

In [ ]:
query_engine = vector_index.as_query_engine(response_mode="refine")

response = query_engine.query("¿Quién fue Picasso?")

display_response(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**`Final Response:`** Pablo Picasso fue un pintor español nacido en Málaga en 1881 y fallecido en París en 1973. Él es considerado uno de los artistas más influyentes del siglo XX y uno de los fundadores del movimiento cubista. Su obra se caracteriza por su uso de colores y formas abstractas y su influencia en el arte contemporáneo.

Picasso fue educado en la Escuela de Bellas Artes de Málaga y más tarde en la Academia de San Fernando de Madrid. En 1904, se trasladó a París, donde se unió a la comunidad de artistas españoles y franceses y desarrolló su estilo cubista. Su obra se exhibió en numerosas exposiciones en Europa y América.

Picasso fue también un prolífico escultor, dibujante y grabador. Sus obras más conocidas incluyen "Les Demoiselles d'Avignon", "Guernica", "The Old Guitarist" y "The Weeping Woman". Picasso también fue un influyente escrit

### 5. Evaluación de los índices

In [ ]:
from llama_index.core.evaluation import FaithfulnessEvaluator

import nest_asyncio
nest_asyncio.apply()

evaluator = FaithfulnessEvaluator(llm=llm)
response_str = response.response
for source_node in response.source_nodes:
    eval_result = evaluator.evaluate(
        response=response_str, contexts=[source_node.get_content()]
    )
    print(str(eval_result.passing))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


True
True


In [ ]:
from llama_index.core.evaluation import RelevancyEvaluator

import nest_asyncio
nest_asyncio.apply()

eval_result = evaluator.evaluate_response(query="¿Cuál es la capital de España?", response=response)
print(str(eval_result))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


query=None contexts=['La capitalidad de Canarias se fija compartidamente en las ciudades de Santa Cruz de Tenerife y Las Palmas de Gran Canaria.</p>\n        <p class="parrafo">La sede del Presidente del Gobierno autónomo alternará entre ambas capitales por períodos legislativos.</p>\n        <p class="parrafo">El Vicepresidente residirá en sede distinta a la del Presidente.</p>\n        <p class="parrafo">Dos. El Parlamento Canario tiene su sede en la ciudad de Santa Cruz de Tenerife.</p>\n        <h5 class="articulo">Artículo cuarto</h5>\n        <p class="parrafo">Uno. A los efectos del presente Estatuto, gozan de la condición política de canarios los ciudadanos españoles que, de acuerdo con las Leyes generales del Estado, tengan vecindad administrativa en cualquiera de los municipios de Canarias.</p>\n        <p class="parrafo">Dos. Como canarios, gozan de los derechos políticos definidos en este Estatuto los ciudadanos españoles residentes en el extranjero que hayan tenido la últi

### 6. Información de dónde sale la respuesta

In [ ]:
response.metadata

{'8da3e3c9-979e-436b-bc41-d376c6e7d54b': {'file_path': '/content/drive/MyDrive/dataPrueba/BOE-A-1982-20821.html',
  'file_name': 'BOE-A-1982-20821.html',
  'file_type': 'text/html',
  'file_size': 119900,
  'creation_date': '2024-02-22',
  'last_modified_date': '2024-02-22',
  'last_accessed_date': '2024-02-22'},
 '4c958731-6f87-48c1-a996-08423ae8f885': {'file_path': '/content/drive/MyDrive/dataPrueba/BOE-A-2007-8444.html',
  'file_name': 'BOE-A-2007-8444.html',
  'file_type': 'text/html',
  'file_size': 182095,
  'creation_date': '2024-02-22',
  'last_modified_date': '2024-02-22',
  'last_accessed_date': '2024-02-22'}}